In [113]:
#This version of the evolutionary algorithm seeks to implement
#elitism, i.e. the keeping of the best solution from previous generations
#and step 6, adding factories from focuses and annexations (mainly the latter)
#add a baseline number of trade factories

#Note: I only added the factories from the baltics, as the other territories
#have little industry
#Also note that factories from annexed territories increase over time as compliance
#grows, this is something that I have not modelled and likely accounts for the 
#remaining underestimation of factories
#Finally, the amount of IC produced is actually higher than they should be
#This is because of problems with the production efficiency calculations

In [159]:
import numpy as np
import time
from statistics import mean

In [115]:
def selection(pop, scores, k=3):
    selection_ix = np.random.randint(len(pop))
    for ix in np.random.randint(0, len(pop), k-1):
        if scores[ix] > scores[selection_ix]:
            selection_ix = ix
    return pop[selection_ix]

def crossover(p1, p2, r_cross):
    c1, c2 = p1.copy(), p2.copy()
    if np.random.rand() < r_cross:
        pt = np.random.randint(1, len(p1)-1)
        c1 = p1[:pt] + p2[pt:]
        c2 = p2[:pt] + p1[pt:]
    return [c1, c2]

def mutation(bitstring, r_mut):
    for i in range(len(bitstring)):
        if np.random.rand() < r_mut:
            bitstring[i] = np.random.randint(0, 4)#1 - bitstring[i]
    return bitstring

In [116]:
#mutation([1] * 10 + [0] * 10, 1/4)

In [117]:
def update_consumergoods(state, toaster_percent):
    match state:
        case 7:
            toaster_percent -= 0.15 #WAR ECO     
        case 30:
            toaster_percent -= 0.05 #construction company
        case 34:
            toaster_percent -= 0.03 #stability bonus
    return round(toaster_percent, 2)

def update_civs(state, civs):
    match state:
        case 8:
            civs += 2
        case 20:
            civs += 2
        case 22:
            civs += 2
        case 25:
            civs += 2
        case 34:
            civs += 13 #annexation of baltics        
    return civs

def update_mils(state, mils):
    match state:
        case 34:
            mils += 8    
    return mils


def update_constructionspeed(state, civ_output_modifier, scenario):
    match scenario:
        case 'one year':
            match state:
                case 0:
                    civ_output_modifier += 0.1 #export focus
                case 5:
                    civ_output_modifier += 0.1
                case 12: #same as half year because we start in 36
                    civ_output_modifier += 0.1 
                case 31:
                    civ_output_modifier += 0.1
                case 52:
                    civ_output_modifier += 0.1
        case 'half year':
            match state:
                case 0:
                    civ_output_modifier += 0.1 #export focus
                case 5:
                    civ_output_modifier += 0.1
                case 12: 
                    civ_output_modifier += 0.1 
                case 33:
                    civ_output_modifier += 0.1
                case 54:
                    civ_output_modifier += 0.1
        case 'current':
            match state:
                case 0:
                    civ_output_modifier += 0.1 #export focus
                case 5:
                    civ_output_modifier += 0.1
                case 16: 
                    civ_output_modifier += 0.1 
                case 36:
                    civ_output_modifier += 0.1
                case 57:
                    civ_output_modifier += 0.1
            
    return round(civ_output_modifier, 2)

def update_miloutput(state, mil_output_modifier, scenario):
    match state: #These are the bonuses related to stability
        case 0:
            mil_output_modifier -= 0.2180
        case 2: 
            mil_output_modifier += 0.05
        case 4: 
            mil_output_modifier += 0.05
        case 8:
            mil_output_modifier += 0.10
        case 9: #At this point we are above 50% stability
            mil_output_modifier += 0.04
        case 10: #popular figurehead
            mil_output_modifier += 0.06 
        case 24: #kill trotsky + new NKVD head
            mil_output_modifier += 0.04
    
    match scenario:
        case 'one year':
            match state:
                case 9:
                    mil_output_modifier += 0.15
                case 14: 
                    mil_output_modifier += 0.15
                case 31:
                    mil_output_modifier += 0.15
                case 52:
                    mil_output_modifier += 0.15
        case 'half year':
            match state:
                case 9:
                    mil_output_modifier += 0.15
                case 14: 
                    mil_output_modifier += 0.15
                case 33:
                    mil_output_modifier += 0.15
                case 54:
                    mil_output_modifier += 0.15
        case 'current':
            match state:
                case 9:
                    mil_output_modifier += 0.15
                case 16: 
                    mil_output_modifier += 0.15
                case 36:
                    mil_output_modifier += 0.15
                case 57:
                    mil_output_modifier += 0.15

    return round(mil_output_modifier, 3)

def update_prodefficiency(state, production_efficiency_cap, scenario):
    match state: #These are related to the 'defense industry' buff in the focus tree
        case 30:
            production_efficiency_cap += 0.05
        case 40:
            production_efficiency_cap += 0.05
    
    match scenario:
        case 'one year':
            match state:
                case 4:
                    production_efficiency_cap += 0.1
                case 13:
                    production_efficiency_cap += 0.1 
                case 26: #construction company american experts
                    production_efficiency_cap += 0.1 
                case 31:
                    production_efficiency_cap += 0.1
                case 52:
                    production_efficiency_cap += 0.1
        case 'half year':
            match state:
                case 4:
                    production_efficiency_cap += 0.1
                case 13: 
                    production_efficiency_cap += 0.1 
                case 26: #construction company american experts
                    production_efficiency_cap += 0.1 
                case 33:
                    production_efficiency_cap += 0.1
                case 54:
                    production_efficiency_cap += 0.1
        case 'current':
            match state:
                case 4:
                    production_efficiency_cap += 0.1
                case 15: 
                    production_efficiency_cap += 0.1 
                case 26: #construction company american experts
                    production_efficiency_cap += 0.1 
                case 36:
                    production_efficiency_cap += 0.1
                case 57:
                    production_efficiency_cap += 0.1

    return round(production_efficiency_cap, 2)

def update_civcost(state, civ_construction_modifier):
    match state:
        case 0:
            civ_construction_modifier += 0.05 #five year plan
        case 7:
            civ_construction_modifier += 0.3
        case 12: 
            civ_construction_modifier += 0.1 #captain of industry
        case 21:
            civ_construction_modifier += 0.05 #construction company
        case 30:
            civ_construction_modifier += 0.05 #construction company upgrade
    return round(civ_construction_modifier, 2)

def update_milcost(state, mil_construction_modifier):
    match state:
        case 7:
            mil_construction_modifier += 0.5
        case 21:
            mil_construction_modifier += 0.05 #construction company
    return round(mil_construction_modifier, 2)

def update_civ_to_milcost(state, civ_to_mil_construction_modifier):
    match state:
        case 7:
            civ_to_mil_construction_modifier += 0.5
    return round(civ_to_mil_construction_modifier, 2)

def update_mil_to_civcost(state, mil_to_civ_construction_modifier):
    match state:
        case 7:
            mil_to_civ_construction_modifier += 0.5
    return round(mil_to_civ_construction_modifier, 2)

def prod_eff_calculations(state, prod_eff, prod_eff_cap, factory_state_array, new_mils):
    factory_count_list = []
    efficiency_list = []
    sum_of_efficiency = 0
    prod_eff = 0
    factory_state_array[state][0] = new_mils
    factory_state_array[state][1] = 0.1
    
    if state == 0:
        factory_state_array[state][1] = 0.5
    
    for i in range(state):
        factory_count_list.append(factory_state_array[i][0])
        efficiency_list.append(factory_state_array[i][1])   
        growth = 0.001 * (prod_eff_cap ** 2 / factory_state_array[i][1]) * 17.5
        factory_state_array[i][1] += growth
        factory_state_array[i][1] = min(factory_state_array[i][1], 1)
        
    num_factories = sum(factory_count_list)
    
    for j in range(len(factory_count_list)):
        sum_of_efficiency += factory_count_list[j] * efficiency_list[j]
    
    try:
        return min(sum_of_efficiency / num_factories, 1)
    except:
        return 0.5 #This should only happen at the start, then eff = 0.5

In [118]:
#Five max_targets: 'rush', 'scale', 'scale_total', 'scale_civs' and 'mils'

def production(actions, max_target, scenario):
    
    #print('actions: ' + str(actions))
    mils = 32
    mils_0 = 32 #keep track of the number of starting factories
    new_mils = 32 #initialize production efficiency
    civs = 45 + 8 #(a baseline number of factories from trade)
    civs_0 = 45 #idem
    total = mils + civs
    total_0 = 32 + 45 #idem
    consumergoods_percentage = 0.40
    consumergoods = np.floor(total * consumergoods_percentage)
    available_civs = np.maximum(int(civs - consumergoods), 0)
    
    #Civ output modifiers are additive (-30% + 15% = -15%), multiplied by state infrastructure
    civ_output = 5 
    civ_output_modifier = 1 #Construction speed
    civ_construction_modifier = 0.7
    mil_construction_modifier = 0.7
    civ_to_mil_construction_modifier = 0.7
    mil_to_civ_construction_modifier = 0.7
    
    mil_output = 4.5
    mil_output_modifier = 1
    prod_eff = 0.5
    production_efficiency_cap = 0.55
    
    state = 0
    state_size_days = 35 #how many days per state
    ic_produced = 0
    ic_total = 0    
    civ_purchase_pool = 0 
    mil_purchase_pool = 0
    civ_to_mil_pool = 0
    mil_to_civ_pool = 0
    infrastructure_modifier = 1.6
    
    factory_state_array = np.empty((57), dtype=object)

    for i in range(len(factory_state_array)):
        factory_state_array[i] = np.zeros(2)
    
    for i in range(len(actions)):
        
        civs = (update_civs(i, civs))
        mils = (update_mils(i, mils))
        consumergoods_percentage = (update_consumergoods(i, consumergoods_percentage))
        available_civs = np.maximum(int(civs - consumergoods), 0)
        civ_output_modifier = (update_constructionspeed(i, civ_output_modifier, scenario))
        mil_output_modifier = (update_miloutput(i, mil_output_modifier, scenario))
        civ_construction_modifier = (update_civcost(i, civ_construction_modifier))
        mil_construction_modifier = (update_milcost(i, mil_construction_modifier))
        civ_to_mil_construction_modifier = (update_civ_to_milcost(i, civ_to_mil_construction_modifier))
        mil_to_civ_construction_modifier = (update_mil_to_civcost(i, mil_to_civ_construction_modifier))
        production_efficiency_cap = (update_prodefficiency(i, production_efficiency_cap, scenario))
        
        ic_produced = mil_output * mils * state_size_days * mil_output_modifier * prod_eff
        prod_eff = prod_eff_calculations(i, prod_eff, production_efficiency_cap, factory_state_array, new_mils)

        if actions[i] == 0: #build mils     
            mil_purchase_pool += (available_civs * civ_output * state_size_days
                                 * infrastructure_modifier * civ_output_modifier
                                 * mil_construction_modifier)
            existing_mils = mils #Used exclusively for production efficiency calculations
            new_mils = np.floor_divide(mil_purchase_pool, 7200) #for production efficiency
            mils += np.floor_divide(mil_purchase_pool, 7200) 
            total += np.floor_divide(mil_purchase_pool, 7200) 
            consumergoods = np.ceil(total * consumergoods_percentage)
            mil_purchase_pool = np.mod(mil_purchase_pool, 7200)
            available_civs = np.maximum(int(civs - consumergoods), 0)
                     
            state += 1
            ic_total += ic_produced
            
        if actions[i] == 1: #build civs         
            civ_purchase_pool += (available_civs * civ_output * state_size_days
                                 * infrastructure_modifier * civ_output_modifier
                                 * civ_construction_modifier)

            new_mils = 0 #Account for annexations here
            civs += np.floor_divide(civ_purchase_pool, 10800) 
            total += np.floor_divide(civ_purchase_pool, 10800) 
            consumergoods = np.ceil(total * consumergoods_percentage)
            civ_purchase_pool = np.mod(civ_purchase_pool, 10800)
            available_civs = np.maximum(int(civs - consumergoods), 0)
            
            state += 1
            ic_total += ic_produced

        if actions[i] == 2: #Convert civs to mils
            civ_to_mil_pool += (available_civs * civ_output * state_size_days
                                 * infrastructure_modifier * civ_output_modifier
                                 * civ_to_mil_construction_modifier)
            new_mils = np.floor_divide(civ_to_mil_pool, 4000)
            civs -= np.floor_divide(civ_to_mil_pool, 4000)
            mils += np.floor_divide(civ_to_mil_pool, 4000)
            civ_to_mil_pool = np.mod(civ_to_mil_pool, 4000)
            consumergoods = np.ceil(total * consumergoods_percentage)
            available_civs = np.maximum(int(civs - consumergoods), 0)
            ic_total += ic_produced
        
        if actions[i] == 3: #Convert mils to civs
            mil_to_civ_pool += (available_civs * civ_output * state_size_days
                                 * infrastructure_modifier * civ_output_modifier
                                 * mil_to_civ_construction_modifier)
            if mils > 0:
                new_mils = 0
                civs += np.floor_divide(mil_to_civ_pool, 9000)
                mils -= np.floor_divide(mil_to_civ_pool, 9000)
                
                if mils < 0:
                    too_many_removed = abs(mils)
                    civs -= too_many_removed
                    mils = 0
                    mil_to_civ_pool = 0
                
                else:
                    mil_to_civ_pool = np.mod(mil_to_civ_pool, 9000)
                consumergoods = np.ceil(total * consumergoods_percentage)
                available_civs = np.maximum(int(civs - consumergoods), 0)
            ic_total += ic_produced
            
    match max_target:
        case 'rush':
            return ((ic_total, mils, civs))
        case 'scale':
            scaled_score = ic_total * np.log2((mils/mils_0))
            return ((scaled_score, mils, civs, ic_total))
        case 'scale_total':
            scaled_score = ic_total * np.log2((mils/mils_0)) * (total/total_0)
            if scaled_score == None:
                scaled_score = 0
            return ((scaled_score, mils, civs, ic_total))
        case 'scale_civs':
            scaled_score = ic_total * np.log2((mils/mils_0)) * np.log2((civs/civs_0))
            if scaled_score == None:
                scaled_score = 0
            return ((scaled_score, mils, civs, ic_total))
        case 'mils':
            return ((mils, civs, ic_total))

In [119]:
#test = ([0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0])
test = [1] * 36 + [0] * 21
#test = [0] * 57
production(test, 'scale', 'half year')

(1479898.3066962948, 191.0, 145.0, 574176.2063801084)

In [120]:
test = [0] * 57
test2 = [1] * 16 + [0] * 41
test3 = [1] * 57

production(test, 'rush', 'half year'), production(test2, 'rush', 'half year'), production(test3, 'rush', 'half year')

#production(test, 'scale'), production(test2, 'scale'), production(test3, 'scale')

#production(test, 'scale_total'), production(test2, 'scale_total'), production(test3, 'scale_total')

#production(test, 'scale_civs'), production(test2, 'scale_civs'), production(test3, 'scale_civs')

((791767.0442336968, 162.0, 74),
 (750434.1166438893, 175.0, 88.0),
 (379047.50497067336, 40, 313.0))

In [175]:
def genetic_algorithm(n_bits, n_pop, n_iter, r_cross, max_target, scenario, mutation_rate):
    
    st = time.time()
    pop = [np.random.randint(0, 4, n_bits).tolist() for _ in range(n_pop)]
    best, best_eval = 0, production(pop[0], max_target, scenario)
    gen_since_last_improvement = 0
    
    for gen in range(n_iter):
        #print('generation ' + str(gen) + ' population: ' + str(pop))
        if gen_since_last_improvement < 10:
            gen_since_last_improvement += 1
            scores = [production(c, max_target, scenario) for c in pop]
            for i in range(n_pop):
                if scores[i] > best_eval:
                    best, best_eval = pop[i], scores[i]
                    gen_since_last_improvement = 0
                    #print("Generation " + str(gen) + ": new best solution found!")
                    #print(best, best_eval)
                
            #ADD ELITISM PLUS ADDITIONAL COMPLETELY RANDOM SAMPLES HERE
        
            selected = [selection(pop, scores) for _ in range(n_pop)]
            children = []
            for i in range(0, n_pop, 2):
                p1, p2 = selected[i], selected[i+1]
                c = crossover(p1, p2, r_cross)
                for child in c:
                    mutated = mutation(child, mutation_rate)
                    children.append(mutated)
            
            if gen > 1:
                del children[-1] #Delete the last element and replace with best solution
                children.append(best)
            pop = children
        else:
            break
    #SMOOTHING TEST
    #best_smoothed = [best[0]]
    #for i in range(1, len(best)-1):
    #    if best[i-1] == best[i+1]:
    #        best_smoothed.append(best[i-1])
    #    else:
    #        best_smoothed.append(best[i])
    #best_smoothed.append(best[-1])
    #best_smoothed_eval = production(best_smoothed, max_target, scenario)
    #if best_smoothed_eval > best_eval:
    #    print("Smoothing : new best solution found!")
    #    print(best_smoothed, best_smoothed_eval)
    #    return [best_smoothed, best_smoothed_eval]
    
    et = time.time()
    elapsed_time = et - st
    print("Time taken: " + str(elapsed_time) + " seconds")
    
    return [best, best_eval]

In [122]:
genetic_algorithm(10, 10, 10, 0.9, 'rush', 'half year', 1 / 57) 

Time taken: 0.03693962097167969 seconds


[[2, 2, 2, 0, 2, 0, 0, 2, 2, 2], (25785.4589755365, 43.0, 45.0)]

In [123]:
#the results for 'scale' seem very similar to those of 'rush'
#It produces 96% of the IC, has 6% more mils and 13% more civs
genetic_algorithm(57,100, 600, 0.9, 'scale', 'half year', 1 / 57)

Time taken: 29.67885971069336 seconds


[[1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  2,
  2,
  2,
  2,
  2,
  2,
  2],
 (1911457.7553933996, 175.0, 53.0, 779801.3586698157)]

In [124]:
#The best strategy for 'scale_total' produces about 81% percent of the IC that the rush strategy does
#It ends up with 18% extra mils and 66% extra civs though
genetic_algorithm(57,100, 600, 0.9, 'scale_total', 'half year', 1 / 57)

Time taken: 42.92240619659424 seconds


[[1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  1,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 (6128881.535692457, 191.0, 121.0, 646991.8871763183)]

In [125]:
#'scale_civs' produces 67% of 'rush' IC, but has 4% more mils and 142% more civs
genetic_algorithm(57,100, 600, 0.9, 'scale_civs', 'half year', 1 / 57)

Time taken: 20.91798686981201 seconds


[[1,
  3,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  3,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  1,
  1,
  1,
  0,
  0,
  1,
  1,
  1,
  1,
  0,
  0,
  1,
  1,
  0,
  0,
  0,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0],
 (2460601.531907159, 162.0, 157.0, 583328.9298726504)]

In [126]:
genetic_algorithm(57,100, 600, 0.9, 'rush', 'half year', 1 / 57)

Time taken: 40.717143297195435 seconds


[[0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  2,
  0,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2],
 (795977.9684608821, 153.0, 33.0)]

In [127]:
genetic_algorithm(57,100, 600, 0.9, 'mils', 'half year', 1 / 57)

Time taken: 35.900052070617676 seconds


[[3,
  3,
  1,
  3,
  3,
  3,
  3,
  3,
  3,
  1,
  3,
  3,
  1,
  1,
  3,
  1,
  3,
  3,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  2,
  2,
  2,
  2],
 (206.0, 113.0, 476190.1209391816)]

In [128]:
#The delay in research does not cause that big of a production loss
#Especially interesting is that going one year ahead is only marginally better
#than half a year, good to know in actual games

test = [0] * 57

(production(test, 'rush', 'one year'), production(test, 'rush', 'half year'), \
 production(test, 'rush', 'current'), production(test, 'rush', 'test'))

((804759.3246453318, 163.0, 74),
 (791767.0442336968, 162.0, 74),
 (765962.3847824389, 161.0, 74),
 (408398.71125558834, 135.0, 74))

In [178]:
#Which parameters should I be tuning?

def hyperparameter_tuning():
    n_pop = [100, 200, 500]
    r_cross = [0.80, 0.90, 1]
    mutation_rate = [0.001, 0.01, 1 / 57, 0.1]
    all_performances_average = []
    
    for i in n_pop:
        for j in r_cross:
            for k in mutation_rate:
                performance_list = [] #Reset this list per different setting
                for l in range(0, 10):
                    result = genetic_algorithm(57, i, 600, j, 'scale_total', 'half year', k)
                    performance_list.append(round(result[1][0], 2))
                print('N_pop: {}, R_cross {}, Mutation_rate {}: '.format(i, j, k) + str(mean(performance_list)))
                all_performances_average.append((mean(performance_list), i, j, k))
    return sorted(all_performances_average, reverse = True)

In [179]:
#tuning = hyperparameter_tuning()
#tuning

Time taken: 29.549476385116577 seconds
Time taken: 46.28278827667236 seconds
Time taken: 48.42827534675598 seconds
Time taken: 30.111562252044678 seconds
Time taken: 20.994133234024048 seconds
Time taken: 26.193993091583252 seconds
Time taken: 19.46025276184082 seconds
Time taken: 46.81636357307434 seconds
Time taken: 43.01304745674133 seconds
Time taken: 20.4213969707489 seconds
N_pop: 100, R_cross 0.8, Mutation_rate 0.001: 5561471.321
Time taken: 38.265586376190186 seconds
Time taken: 26.601184606552124 seconds
Time taken: 44.2306022644043 seconds
Time taken: 36.22605013847351 seconds
Time taken: 28.2087721824646 seconds
Time taken: 42.996411085128784 seconds
Time taken: 29.30710792541504 seconds
Time taken: 34.63320255279541 seconds
Time taken: 44.45432639122009 seconds
Time taken: 33.678114891052246 seconds
N_pop: 100, R_cross 0.8, Mutation_rate 0.01: 6037965.603
Time taken: 40.18059849739075 seconds
Time taken: 29.942201852798462 seconds
Time taken: 35.8885235786438 seconds
Time t

Time taken: 48.77134084701538 seconds
Time taken: 53.205827474594116 seconds
Time taken: 82.86727738380432 seconds
Time taken: 65.70449686050415 seconds
N_pop: 200, R_cross 0.9, Mutation_rate 0.017543859649122806: 6124779.81
Time taken: 32.52871322631836 seconds
Time taken: 36.510149240493774 seconds
Time taken: 27.5513117313385 seconds
Time taken: 42.96400284767151 seconds
Time taken: 47.60376977920532 seconds
Time taken: 28.234262704849243 seconds
Time taken: 45.50650072097778 seconds
Time taken: 33.33469605445862 seconds
Time taken: 38.96735167503357 seconds
Time taken: 37.24354362487793 seconds
N_pop: 200, R_cross 0.9, Mutation_rate 0.1: 5034034.569
Time taken: 48.707146644592285 seconds
Time taken: 58.70849823951721 seconds
Time taken: 66.0233154296875 seconds
Time taken: 58.124587059020996 seconds
Time taken: 67.65580010414124 seconds
Time taken: 58.09580063819885 seconds
Time taken: 53.4290976524353 seconds
Time taken: 48.90861535072327 seconds
Time taken: 70.34145164489746 seco

[(6151354.4969999995, 500, 0.8, 0.017543859649122806),
 (6141190.765, 500, 0.8, 0.01),
 (6139304.692, 500, 0.9, 0.017543859649122806),
 (6138229.692, 500, 1, 0.01),
 (6137890.265, 500, 1, 0.017543859649122806),
 (6131516.4059999995, 500, 0.9, 0.01),
 (6126966.847, 200, 1, 0.017543859649122806),
 (6124779.81, 200, 0.9, 0.017543859649122806),
 (6095898.870999999, 200, 0.9, 0.01),
 (6088720.391, 200, 0.8, 0.017543859649122806),
 (6058727.3100000005, 500, 0.8, 0.001),
 (6049658.739, 500, 1, 0.001),
 (6048796.828, 100, 1, 0.017543859649122806),
 (6046341.458, 100, 0.8, 0.017543859649122806),
 (6045110.3889999995, 200, 0.8, 0.01),
 (6037965.603, 100, 0.8, 0.01),
 (6037130.596, 100, 0.9, 0.017543859649122806),
 (6034854.692, 200, 1, 0.01),
 (6030770.13, 100, 1, 0.01),
 (6021191.24, 100, 0.9, 0.01),
 (5987066.933, 500, 0.9, 0.001),
 (5947535.682, 200, 0.8, 0.001),
 (5891037.638, 200, 1, 0.001),
 (5854684.155, 200, 0.9, 0.001),
 (5700835.669, 100, 0.9, 0.001),
 (5605893.064, 100, 1, 0.001),
 (5

In [131]:
#genetic_algorithm(57,500, 600, 0.9, 'scale_civs', 'half year')
#about 150 seconds with a population of 500

In [180]:
#genetic_algorithm(57,200, 600, 0.9, 'scale_civs', 'half year', 1 / 57)
#about 60 seconds with a population of 300

In [133]:
#genetic_algorithm(57,100, 600, 0.9, 'scale_civs', 'half year')
#about 35 seconds with a population of 1

In [181]:
tuning_data = [(6151354.4969999995, 500, 0.8, 0.017543859649122806),
 (6141190.765, 500, 0.8, 0.01),
 (6139304.692, 500, 0.9, 0.017543859649122806),
 (6138229.692, 500, 1, 0.01),
 (6137890.265, 500, 1, 0.017543859649122806),
 (6131516.4059999995, 500, 0.9, 0.01),
 (6126966.847, 200, 1, 0.017543859649122806),
 (6124779.81, 200, 0.9, 0.017543859649122806),
 (6095898.870999999, 200, 0.9, 0.01),
 (6088720.391, 200, 0.8, 0.017543859649122806),
 (6058727.3100000005, 500, 0.8, 0.001),
 (6049658.739, 500, 1, 0.001),
 (6048796.828, 100, 1, 0.017543859649122806),
 (6046341.458, 100, 0.8, 0.017543859649122806),
 (6045110.3889999995, 200, 0.8, 0.01),
 (6037965.603, 100, 0.8, 0.01),
 (6037130.596, 100, 0.9, 0.017543859649122806),
 (6034854.692, 200, 1, 0.01),
 (6030770.13, 100, 1, 0.01),
 (6021191.24, 100, 0.9, 0.01),
 (5987066.933, 500, 0.9, 0.001),
 (5947535.682, 200, 0.8, 0.001),
 (5891037.638, 200, 1, 0.001),
 (5854684.155, 200, 0.9, 0.001),
 (5700835.669, 100, 0.9, 0.001),
 (5605893.064, 100, 1, 0.001),
 (5561471.321, 100, 0.8, 0.001),
 (5170051.133, 500, 1, 0.1),
 (5061097.736, 500, 0.9, 0.1),
 (5034034.569, 200, 0.9, 0.1),
 (4979460.757, 100, 0.8, 0.1),
 (4969699.640000001, 500, 0.8, 0.1),
 (4955521.856, 200, 0.8, 0.1),
 (4935814.575, 100, 0.9, 0.1),
 (4921377.552, 200, 1, 0.1),
 (4915185.546, 100, 1, 0.1)]